In [6]:
from osgeo import gdal, ogr
import os
from os.path import isfile, isdir, join
import re
import glob
import numpy
import rasterio as rio
import pandas as pd
import numpy as np
from rasterio.mask import mask
from datetime import datetime
import itertools

# Model building
#from sklearn import linear_model, preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, KFold
#from sklearn import feature_selection, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
#from sklearn.preprocessing import OrdinalEncoder
import zipfile
import shutil

import time

import sys

In [ ]:
# From pyeo
def convert_shapefile2array(trainingfnm,img_ds_str,nodata_value,options=["ATTRIBUTE=id"]):

    img_ds = gdal.Open(img_ds_str)

    trng_ds=ogr.Open(trainingfnm)
    trng_layer=trng_ds.GetLayer()
    trng_rds = gdal.GetDriverByName("MEM").Create("",img_ds.RasterXSize,img_ds.RasterYSize,1,gdal.GDT_Int16)
    trng_rds.SetGeoTransform(img_ds.GetGeoTransform())
    trng_rds.SetProjection(img_ds.GetProjection())
    band = trng_rds.GetRasterBand(1)
    band.SetNoDataValue(-1)
    tra = numpy.zeros((img_ds.RasterYSize,img_ds.RasterXSize), dtype = numpy.int32)
    tra[:]=nodata_value
    trng_rds.GetRasterBand(1).WriteArray(tra)
    gdal.RasterizeLayer(trng_rds, [1], trng_layer, burn_values=[nodata_value],options=options)
    tra=trng_rds.GetRasterBand(1).ReadAsArray()
    trng_ds,trng_rds=None,None
    return tra

# Set working directory
wd = sys.argv[1]
os.chdir(wd)

# Create list of Sentinel tiles in working directory (files should have training data
# and at least one tif of the scene (need the dimensions))
scenes = [f for f in os.listdir(wd) if re.search(r"\AS2[AB]_2", f) and zipfile.is_zipfile(f)]

In [1]:
master_csv = join(wd, 'master.csv')

if os.path.exists(master_csv):
    master_df = pd.read_csv(master_csv)
    
else:
    master_df = pd.DataFrame(columns = ['scene',
                                        'pre_date',
                                        'post_date',
                                        'interval_days',
                                        'x_pos',
                                        'y_pos',
                                        'pre_b4',
                                        'pre_b8',
                                        'pre_b12',
                                        'post_b4',
                                        'post_b8',
                                        'post_b12',
                                        'pre_NBR',
                                        'post_NBR',
                                        'dNBR',
                                        'burn_class'])

In [5]:
master_df = master_df.drop(columns=['Unnamed: 0'])
master_df

,scene,pre_date,post_date,interval_days,x_pos,y_pos,pre_b4,pre_b8,pre_b12,post_b4,post_b8,post_b12,pre_NBR,post_NBR,dNBR,burn_class,pre_month
0,S2A_20191221_S2A_20200130_T55HGU_R030,2019-12-21,2020-01-30,40,4664,64,885,1926,791,732,1638,634,0.417740,0.441901,0.024161,3,12.0
1,S2A_20191221_S2A_20200130_T55HGU_R030,2019-12-21,2020-01-30,40,4665,64,889,2076,782,744,1602,631,0.452764,0.434841,-0.017923,3,12.0
2,S2A_20191221_S2A_20200130_T55HGU_R030,2019-12-21,2020-01-30,40,4664,65,864,1916,728,724,1604,569,0.449319,0.476300,0.026981,3,12.0
3,S2A_20191221_S2A_20200130_T55HGU_R030,2019-12-21,2020-01-30,40,4665,65,861,2060,687,707,1664,541,0.499818,0.509297,0.009479,3,12.0
4,S2A_20191221_S2A_20200130_T55HGU_R030,2019-12-21,2020-01-30,40,4666,65,877,1962,687,731,1624,541,0.481314,0.500231,0.018917,3,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5392841,S2A_20191201_S2A_20191211_T55HGA_R030,2019-12-01,2019-12-11,10,3790,9547,8140,9328,7160,5856,6328,5030,0.131490,0.114281,-0.017209,2,12.0
5392842,S2A_20191201_S2A_20191211_T55HGA_R030,2019-12-01,2019-12-11,10,3791,9547,7828,8920,7061,5952,6392,5128,0.116326,0.109722,-0.006603,2,12.0
5392843,S2A_20191201_S2A_20191211_T55HGA_R030,2019-12-01,2019-12-11,10,3792,9547,7612,8328,7061,6144,6528,5128,0.082332,0.120110,0.037778,2,12.0
5392844,S2A_20191201_S2A_20191211_T55HGA_R030,2019-12-01,2019-12-11,10,3790,9548,8120,9112,7160,5860,6328,5030,0.119961,0.114281,-0.005680,2,12.0


In [8]:
for scene in scenes:
    
    with zipfile.ZipFile(scene, 'r') as zip:
        zip.extractall()
    
    folder = scene[:-4]
    img_ds = join(folder, [f for f in os.listdir(folder) if re.search(r"[0-9].tif\Z", f)][0])
    shp = join(folder, 'training/training.shp')

    burn_class = convert_shapefile2array(shp,img_ds,-9999)
    
    with rio.open(img_ds) as src:
        pre_4 = src.read(1)
        pre_8 = src.read(2)
        pre_12 = src.read(3)
        po_4 = src.read(4)
        po_8 = src.read(5)
        po_12 = src.read(6)
        shape = src.shape
        src.close()
    
    pre_NBR = (pre_8 - pre_12)/(pre_8 + pre_12)
    post_NBR = (po_8 - po_12)/(po_8 + po_12)
    dNBR = post_NBR - pre_NBR
    
    training_arrays = {
        'pre_b4': np.ravel(pre_4),
        'pre_b8': np.ravel(pre_8),
        'pre_b12': np.ravel(pre_12),
        'post_b4': np.ravel(po_4),
        'post_b8': np.ravel(po_8),
        'post_b12': np.ravel(po_12),
        'pre_NBR': np.ravel(pre_NBR),
        'post_NBR': np.ravel(post_NBR),
        'dNBR': np.ravel(dNBR),
        'burn_class': np.ravel(burn_class)
    }

    df = pd.DataFrame.from_dict(training_arrays)
    
    # Need to add pos before removing no data vals
    df['x_pos'] = list(range(1,shape[1]+1)) * shape[0]
    lst = range(1,shape[0]+1)
    df['y_pos'] = list(itertools.chain.from_iterable(itertools.repeat(x, shape[0]) for x in lst))
    
    df = df[df.burn_class != -9999]
    df['scene'] = folder
    df['pre_date'] = datetime.strptime(folder.rsplit('_')[1], '%Y%m%d')
    df['post_date'] = datetime.strptime(folder.rsplit('_')[3], '%Y%m%d')
    df['interval_days'] = (df['post_date'] - df['pre_date']).apply(lambda x: x.days)
    df['pre_month'] = df['pre_date'].apply(lambda x: x.month)
    
    master_df = pd.concat([master_df, df]).reset_index(drop=True)
    
master_df.to_csv(master_csv, index=False)

In [14]:
os.path.isdir(folder)

True

In [7]:
scenes

['S2A_20191221_S2A_20200130_T55HGV_R030.zip']

In [18]:
# Select predictor variables
X = master_df[['pre_NBR', 'post_NBR', 'dNBR', 'interval', 'pre_month']]
# Want to predict award
Y = master_df['burn_class']
# Create train test split using 20% of the rows to test the model and stratified
trainX, testX, trainY, testY=train_test_split(X, Y, stratify=Y, test_size=0.2)

# Create Random Forest classifier model
rf_interval = RandomForestClassifier(n_jobs=-1, oob_score=True)#, n_estimators=10000, min_samples_leaf=15, max_depth=30)
rf_interval.fit(trainX,trainY)
train_score = rf_interval.score(trainX, trainY)
test_score = rf_interval.score(testX, testY)
oob_score = rf_interval.oob_score_
print('Accuracy score for training data is: {:4.3f}'.format(train_score))
print('Accuracy score for test data: {:4.3f}'.format(test_score))
print('The Oob score is: {:4.3f}'.format(oob_score))

Accuracy score for training data is: 1.000
Accuracy score for test data: 0.838
The Oob score is: 0.836


In [19]:
# Confusion matrix
predicted = rf_interval.predict(testX)
y_actu = pd.Series(list(testY), name='Actual')
y_pred = pd.Series(predicted, name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred)
df_confusion

Predicted,burned,no data,unburned
Actual,,,
burned,34448,11339,6485
no data,6073,207723,56284
unburned,3543,40178,396838


In [20]:
# KFold cross validation of the model
crossvalidation = KFold(6, shuffle=True)
scores = cross_val_score(rf_interval, X, Y, scoring='accuracy',cv=crossvalidation, n_jobs=-1)
print('X validation score for Random Forest: {:5.4f}'.format(np.mean(scores)))
print('All scores: \n', scores)

X validation score for Random Forest:   nan
All scores: 
 [       nan        nan        nan 0.83758783        nan        nan]


In [27]:
import joblib
joblib.dump(rf_interval, 'rf_preNBR_postNBR_dNBR_interval.pkl')

ImportError: cannot import name 'joblib' from 'sklearn.externals' (C:\Users\benco\anaconda3\envs\geo_env\lib\site-packages\sklearn\externals\__init__.py)

In [25]:
# Select predictor variables
X = master_df[['pre_NBR', 'post_NBR', 'dNBR']]
# Want to predict award
Y = master_df['burn_class']
# Create train test split using 20% of the rows to test the model and stratified
trainX, testX, trainY, testY=train_test_split(X, Y, stratify=Y, test_size=0.2)

# Create Random Forest classifier model
rf_pixelvals = RandomForestClassifier(n_jobs=-1, oob_score=True)#, n_estimators=10000, min_samples_leaf=15, max_depth=30)
rf_pixelvals.fit(trainX,trainY)
train_score = rf_pixelvals.score(trainX, trainY)
test_score = rf_pixelvals.score(testX, testY)
oob_score = rf_pixelvals.oob_score_
print('Accuracy score for training data is: {:4.3f}'.format(train_score))
print('Accuracy score for test data: {:4.3f}'.format(test_score))
print('The Oob score is: {:4.3f}'.format(oob_score))

Accuracy score for training data is: 1.000
Accuracy score for test data: 0.738
The Oob score is: 0.735


In [26]:
# Confusion matrix
predicted = rf_pixelvals.predict(testX)
y_actu = pd.Series(list(testY), name='Actual')
y_pred = pd.Series(predicted, name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred)
df_confusion

Predicted,1,2,3
Actual,,,
1,27391,15378,9503
2,10305,165588,94187
3,5887,64731,369941


In [28]:
import joblib
joblib.dump(rf_pixelvals, 'rf_preNBR_postNBR_dNBR.pkl')

['rf_preNBR_postNBR_dNBR.pkl']

# Adding dates and time interval between scenes

In [3]:
# Set working directory
wd = 'C:\\Users\\benco\\OneDrive\\Documents\\Uni\\dissertation\\training_aggregation'
os.chdir(wd)

master_csv = join(wd, 'master.csv')
master_df = pd.read_csv(master_csv)

In [36]:
def get_predate(string):
    return datetime.strptime(string.rsplit('_')[1], '%Y%m%d')
    
def get_postdate(string):
    return datetime.strptime(string.rsplit('_')[3], '%Y%m%d')

master_df['predate'] = master_df['scene'].apply(get_predate)
master_df['postdate'] = master_df['scene'].apply(get_postdate)
master_df['interval'] = (master_df['postdate'] - master_df['predate']).apply(lambda x: x.days)

In [37]:
master_df['pre_month'] = master_df['predate'].apply(lambda x: x.month)

In [16]:
def get_burn_class(num):
    if num == 1:
        return 'burned'
    if num == 2:
        return 'no data'
    if num == 3:
        return 'unburned'
    
master_df['burn_class'] = master_df['burn_class'].apply(get_burn_class)
master_df

,scene,pre_NBR,post_NBR,dNBR,burn_class,predate,postdate,interval,pre_month
0,S2A_20191121_S2A_20191201_T55HGA_R030,-0.179257,-0.168421,0.010836,unburned,2019-11-21,2019-12-01,10,11
1,S2A_20191121_S2A_20191201_T55HGA_R030,-0.187158,-0.174001,0.013157,unburned,2019-11-21,2019-12-01,10,11
2,S2A_20191121_S2A_20191201_T55HGA_R030,-0.183329,-0.172610,0.010719,unburned,2019-11-21,2019-12-01,10,11
3,S2A_20191121_S2A_20191201_T55HGA_R030,-0.171952,-0.159820,0.012133,unburned,2019-11-21,2019-12-01,10,11
4,S2A_20191121_S2A_20191201_T55HGA_R030,-0.172533,-0.159820,0.012714,unburned,2019-11-21,2019-12-01,10,11
...,...,...,...,...,...,...,...,...,...
3814550,S2A_20191221_S2A_20200130_T55HGV_R030,0.143426,0.453763,0.310337,unburned,2019-12-21,2020-01-30,40,12
3814551,S2A_20191221_S2A_20200130_T55HGV_R030,0.166128,0.464512,0.298384,unburned,2019-12-21,2020-01-30,40,12
3814552,S2A_20191221_S2A_20200130_T55HGV_R030,0.213775,0.466667,0.252891,unburned,2019-12-21,2020-01-30,40,12
3814553,S2A_20191221_S2A_20200130_T55HGV_R030,0.210183,0.464727,0.254545,unburned,2019-12-21,2020-01-30,40,12


In [31]:
master_df = pd.read_csv("master_old.csv")

C:\Users\benco\anaconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [40]:
master_df

,scene,pre_NBR,post_NBR,dNBR,burn_class,predate,postdate,interval,pre_month
0,S2A_20191121_S2A_20191201_T55HGA_R030,-0.179257,-0.168421,0.010836,3,2019-11-21,2019-12-01,10,11
1,S2A_20191121_S2A_20191201_T55HGA_R030,-0.187158,-0.174001,0.013157,3,2019-11-21,2019-12-01,10,11
2,S2A_20191121_S2A_20191201_T55HGA_R030,-0.183329,-0.172610,0.010719,3,2019-11-21,2019-12-01,10,11
3,S2A_20191121_S2A_20191201_T55HGA_R030,-0.171952,-0.159820,0.012133,3,2019-11-21,2019-12-01,10,11
4,S2A_20191121_S2A_20191201_T55HGA_R030,-0.172533,-0.159820,0.012714,3,2019-11-21,2019-12-01,10,11
...,...,...,...,...,...,...,...,...,...
5447490,S2B_20200608_S2A_20200623_T54LXK_R102,-0.435019,0.076741,0.511760,2,2020-06-08,2020-06-23,15,6
5447491,S2B_20200608_S2A_20200623_T54LXK_R102,-0.324900,0.068353,0.393253,2,2020-06-08,2020-06-23,15,6
5447492,S2B_20200608_S2A_20200623_T54LXK_R102,-0.331295,0.109640,0.440936,2,2020-06-08,2020-06-23,15,6
5447493,S2B_20200608_S2A_20200623_T54LXK_R102,-0.345908,0.158803,0.504711,2,2020-06-08,2020-06-23,15,6


In [41]:
master_df.to_csv("master.csv", index=False)